In [158]:
import pandas as pd
import json
from glob import glob
import os.path as osp
import datetime
import wordcloud
from io import StringIO
import stop_words
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import re

In [159]:
pd.set_option('display.max_colwidth', 5000)

In [160]:
media = 'France 2'
day = '2022-05-21'
edition = 'jt-de-20h'

In [171]:
media = 'TF1'
day = '2022-05-25'
edition = 'le-20h'

In [172]:
subjects = []
day_files = glob(f'../data-news-json/media={media}/year={day[:4]}/month={int(day[5:7])}/day={int(day[8:10])}/*.json')
for day_file in day_files:    
    for line in open(day_file):
        subject = json.loads(line)
        subject['media'] = media.lower().replace(' ', '')
        subjects.append(subject)

In [173]:
data = pd.DataFrame(subjects)

In [174]:
data.head()

,title,description,date,order,presenter,authors,editor,editorDeputy,url,urlTvNews,containsWordGlobalWarming,media
0,"Abricots, cerises, pêches : les fruits d'été en avance sur les étals","Les fruits d'été sont déjà en vente sur les étals avec trois semaines d'avance, à cause de très fortes chaleurs. La météo contraint les agriculteurs à récolter et à vendre leurs produits beaucoup plus tôt que d'habitude. Reportage dans les Yvelines. Des abricots, des pêches, des melons et même des cerises sont déjà arrivés sur les étals quelques jours, voire quelques semaines avant la saison. Les clients en profitent. Les prix ne sont pas plus élevés qu'en pleine saison. Alors, depuis début mai, sur les étals de ce marché en région parisienne, les fruits d'été s'arrachent. On constate la même chose pour Marc Surgis, qui n'a jamais vendu aussi tôt dans la saison sa production de cerises. Ce maraîcher tient à nous montrer les arbres de son verger. Ils sont aussi chargés qu'en plein été. ""Grosso modo, on a entre 25 et 35 jours d'avance. C'est énorme avec ce mois de mai qui a été très chaud"", avoue-t-il. Le souci, c'est qu'il n'a que quelques jours seulement pour tout récolter. En outre, sa production arrive sur le marché au même moment où les cerises du sud de la France. Un problème pour les producteurs du Vaucluse qui, d'habitude, sont les premiers à vendre. Les prix ont tendance à baisser. Alors, ils vendront leur production deux fois moins chère qu'en 2021. En revanche, c'est une aubaine pour les consommateurs qui se régalent des fruits de qualité à des tarifs raisonnables, un peu plus tôt dans la saison.",2022-05-25T15:47:09.313Z,0,,[],,[],https://www.tf1info.fr/conso/video-abricots-cerises-peches-les-fruits-d-ete-en-avance-sur-les-etals-2221014.html,https://www.tf1info.fr/emission/le-20h-11001/extraits,False,tf1
1,Pont de l’Ascension : c’est parti pour quatre jours de vacances,"Il y a du monde sur les routes et dans les gares. Douze millions de Français ont pris le pont de l'Ascension. Les premiers chanceux sont arrivés notamment au Pays Basque et autour du bassin d'Arcachon. Les premiers vacanciers de l'Ascension sont bien arrivés à Biarritz (Pyrénées-Atlantiques). À peine installés, certains ont déjà les pieds dans l'eau. ""Ce n'est pas encore le temps idéal, mais ça fait du bien, ça fait plaisir"", un couple. tongues, glaces, jeux sur le sable... L'été semble presque là. C'est un avant-goût des vacances. Certains ont programmé de se balader, de prendre du bon air, de sortir du quotidien et surtout de profiter de ce week-end prolongé. D'autres ont laissé les plages pour un tour de petit train. L'option idéale pour voir toute la ville sans perdre de temps. Avec quelques jours de vacances seulement, l'emploi du temps est chargé pour un couple venu des Sables-d'Olonne. À quelques kilomètres de là, les terrasses ont aussi commencé à se remplir à Anglet. Les professionnels du tourisme ont le sourire. C'est le cas de Pascal Marien, restaurateur dans la région. Il se veut optimiste avant ce long week-end. De nombreux hébergements affichent aussi complet jusqu'à dimanche sur la côte basque. Du côté du bassin d'Arcachon, les quinze chambres d'un hôtel seront occupées dès jeudi. Seuls les nuages pourraient gâcher le début des vacances demain avant de se dissiper dès vendredi.",2022-05-25T15:47:09.683Z,0,,[],,[],https://www.tf1info.fr/voyages/video-pont-de-l-ascension-c-est-parti-pour-quatre-jours-de-vacances-2221011.html,https://www.tf1info.fr/emission/le-20h-11001/extraits,False,tf1
2,Algues vertes : des aides pour les agriculteurs bretons qui utilisent moins d'engrais,"C'est un phénomène bien connu sur le littoral breton : les algues vertes reviennent chaque année. L’État veut proposer des aides aux agriculteurs pour réduire leur utilisation d'engrais. Sur cette plage de Saint-Michel-en-Grève (Côtes-d'Armor), aucune algue ce matin, mais les habitants saluent l'initiative : pour mieux lutter contre le fléau des algues vertes, l’État veut propose

In [175]:
data = data[[edition in x for x in data.urlTvNews]]

In [176]:
data.columns

Index(['title', 'description', 'date', 'order', 'presenter', 'authors',
       'editor', 'editorDeputy', 'url', 'urlTvNews',
       'containsWordGlobalWarming', 'media'],
      dtype='object')

In [177]:
data[['date', 'order', 'title', 'description', 'presenter', 'authors', 'editor', 'editorDeputy', 'url', 'urlTvNews']
    ].sort_values('order')

,date,order,title,description,presenter,authors,editor,editorDeputy,url,urlTvNews
0,2022-05-25T15:47:09.313Z,0,"Abricots, cerises, pêches : les fruits d'été en avance sur les étals","Les fruits d'été sont déjà en vente sur les étals avec trois semaines d'avance, à cause de très fortes chaleurs. La météo contraint les agriculteurs à récolter et à vendre leurs produits beaucoup plus tôt que d'habitude. Reportage dans les Yvelines. Des abricots, des pêches, des melons et même des cerises sont déjà arrivés sur les étals quelques jours, voire quelques semaines avant la saison. Les clients en profitent. Les prix ne sont pas plus élevés qu'en pleine saison. Alors, depuis début mai, sur les étals de ce marché en région parisienne, les fruits d'été s'arrachent. On constate la même chose pour Marc Surgis, qui n'a jamais vendu aussi tôt dans la saison sa production de cerises. Ce maraîcher tient à nous montrer les arbres de son verger. Ils sont aussi chargés qu'en plein été. ""Grosso modo, on a entre 25 et 35 jours d'avance. C'est énorme avec ce mois de mai qui a été très chaud"", avoue-t-il. Le souci, c'est qu'il n'a que quelques jours seulement pour tout récolter. En outre, sa production arrive sur le marché au même moment où les cerises du sud de la France. Un problème pour les producteurs du Vaucluse qui, d'habitude, sont les premiers à vendre. Les prix ont tendance à baisser. Alors, ils vendront leur production deux fois moins chère qu'en 2021. En revanche, c'est une aubaine pour les consommateurs qui se régalent des fruits de qualité à des tarifs raisonnables, un peu plus tôt dans la saison.",,[],,[],https://www.tf1info.fr/conso/video-abricots-cerises-peches-les-fruits-d-ete-en-avance-sur-les-etals-2221014.html,https://www.tf1info.fr/emission/le-20h-11001/extraits
29,2022-05-25T15:47:10.288Z,0,Risque d’effondrement à Bordeaux : plusieurs immeubles du centre-ville évacués,"À Bordeaux, plusieurs immeubles menaçant de s'effondrer ont été évacués en urgence. La mairie a pris cette décision après avoir été alertée par un ouvrier qui a constaté d'importantes fissures sur une façade en plein centre-ville. À la hâte, ils prennent tout ce qu'ils peuvent emporter avec eux. Sommés d'évacuer en moins de 24 heures, ces habitants quittent leur immeuble, car il menace de s'effondrer. En atteste, les nombreuses fissures visibles sur la façade. Nous sommes sur la Place Gambetta, en plein centre de Bordeaux. Au rez-de-chaussée du bâtiment, les commerçants, eux aussi, ont dû stopper leurs activités. Signalées à la mairie il y a quelques jours, ces fissures ont été jugées inquiétantes par un expert du tribunal administratif. À l'intérieur, un mur porteur serait sur le point de rompre, d'où l'obligation d'évacuer en urgence avant 18 heures. Un déficit d'entretien de l'immeuble peut conduire au pire. Il y a tout juste onze mois, à quelques centaines de mètres de là, rue de la Rousselle, deux immeubles s'étaient effondrés, faisant trois blessés, dont un grave. Depuis, un audit a été lancé pour repérer tous les immeubles vétustes de la ville : 5% seraient à risque, soit 7 000 logements. T F1 | Reportage C. Adriaens-allemand, Y. Chambon",,"[C. Adriaens-allemand, Y. Chambon]",,[],https://www.tf1info.fr/societe/video-risque-d-effondrement-a-bordeaux-plusieurs-immeubles-du-centre-ville-evacues-2221008.html,https://www.tf1info.fr/emission/le-20h-11001/extraits
28,2022-05-25T15:47:12.856Z,0,Immobilier : de nombreuses erreurs épinglées dans les diagnostics énergétiques,"Le diagnostic de performance énergétique est obligatoire pour vendre ou louer un logement. Mais l'association 60 Millions de consommateurs a mené des tests pour en vérifier la fiabilité... et épingle de nombreuses erreurs. Le diagnostic de performance énergétique (DPE), une lettre allant de A à G pour classer les logements, est obligatoire pour vendre ou louer un bien. Comme son nom l’indique, cette lettre détermine la performance énergétique d’un bien mais aussi son prix. En juillet 2021, ce dia

In [178]:
len(data)

17

In [148]:
export_fields = ['order', 'title', 'description', 'url']

In [150]:
data[export_fields].sort_values('order').head()

,order,title,description,url
8,1,Allemagne : une tornade spectaculaire a balayé l'ouest du pays,"Une tornade spectaculaire a balayé, vendredi 20 mai, l'ouest de l'Allemagne, en Rhénanie-Palatinat. C'est la commune de Paderborn qui a été principalement touchée, et les dégâts sont considérables.Réfugiés sous un abribus, ils sont pris par surprise par la tornade. Des vents à plus de 200 km/h balaient le centre-ville de Paderborn, vendredi 20 mai, dans l'ouest de l'Allemagne. Les habitants se barricadent en urgence car, dehors, les débris pleuvent, et les arbres s'écroulent. Un phénomène rarissime dans le pays, survenu après plusieurs jours de forte chaleur. Dans la ville de Lippstadt, une autre tornade se forme. La personne qui filme les images s'éloigne juste à temps. Quelques minutes plus tard, le magasin est entièrement dévasté. Des dégâts matériels de plusieurs milliers d'euros Scène de chaos après le passage de la tornade, les riverains sont abasourdis. ""C'est un scénario de film de guerre"", dit une habitante. Des arbres ont même été déracinés par la tempête. En tout, 43 personnes ont été blessées, et les dégâts matériels sont évalués à plusieurs millions d'euros. Ces intempéries record ont touché tout l'ouest du pays. Un homme de 38 ans est décédé.",https://www.francetvinfo.fr/monde/europe/allemagne/allemagne-une-tornade-spectaculaire-a-balaye-l-ouest-du-pays_5151460.html
7,2,Tornades : plus fréquentes et plus violentes avec le réchauffement climatique ?,"La tornade, provoquée par la rencontre de l'air chaud et de l'air froid, est encore un phénomène rare en Europe. Mais ce phénomène extrême se multipliera-t-il à l’avenir avec le réchauffement climatique ?C’est un spectacle effrayant que l’on observe aujourd’hui plus de six mois par an dans tout le centre des États-Unis. Des tornades d’une puissance parfois exceptionnelle comme l’une qui a lieu samedi 21 mai dans le Michigan et qui ne laisse derrière elle que des toitures arrachées et des voitures retournées. Ces scènes habituelles en Amérique du Nord pourraient-elles aussi le devenir en Europe, dès le printemps ? C’était impensable auparavant mais les tornades en République Tchèque l’année dernière et en Allemagne cette semaine ont changé la donne. Les climatologues divisés Pour que des nuages d’orage engendrent des tornades, il faut la rencontre de masse d’air chaud au sol et d’air beaucoup plus froid en altitude, poussé par des vents tournoyants. Ces phénomènes deviendront-ils plus nombreux et plus violents avec le dérèglement climatique ? La question fait encore débat chez les climatologues. ""En théorie, on devrait avoir des oranges plus violentes, des conditions convectives plus violentes et donc des situations plus favorables aux tornades. Cela, c’est la théorie pas encore observée"", explique Robert Vautard, climatologue et membre du Giec.",https://www.francetvinfo.fr/meteo/orages/tornades-plus-frequentes-et-plus-violentes-avec-le-rechauffement-climatique_5151430.html
3,3,"Sécheresse : en Ardèche, départs d'incendies précoces pour la saison","En France, on totalise en ce printemps presque 40 jours consécutifs au-dessus des normales de saison, avec pour conséquence notamment une sécheresse précoce, en région Rhône-Alpes et en Ardèche particulièrement. Les premiers départs de feu ont été enregistrés.Une reprise de feu et un combat acharné contre les flammes. Depuis plus de 24 heures, les pompiers d'Ardèche sont sur le front. Deux incendies se sont déclarés dans le département. Des feux de forêt inhabituels pour un mois de mai. Les pompiers redoutent un été à haut risque. Les départs d'incendie sont très précoces pour la saison. Pour les habitants, le phénomène interroge. Crainte d'incendies à répétition Depuis le mois de janvier, les épisodes pluvieux sont rares, ils ont baissé de 40 % par rapport à l'an dernier. ""C'est inquiétant pour la flore, pour la faune également qui va souffrir, et puis pour les activités touristiques ça va être compliqué"", s'inquiète Didie

In [151]:
data[export_fields].sort_values('order').head().to_csv('data/france2_20h_20220521.csv', index=False)

In [123]:
scores = []
for title, description, order in sorted(zip(data.title, data.description, data.order), key=lambda x: x[2]):
    score = {}
    print(f'{order}: {title}')
    score['title'] = title
    score['description'] = description
    print("Climat related ?")
    score['climat'] = int(input())
    if score['climat']:
        print(description)
        print("Climat score ?")
        score['climat_score'] = input()
        if score['climat_score'] == '0':
            score['climat'] = 0
            score['climat_score'] = None
    else:
        score['climat_score'] = None
    scores.append(score)

1: Allemagne : une tornade spectaculaire a balayé l'ouest du pays
Climat related ?
1
Une tornade spectaculaire a balayé, vendredi 20 mai, l'ouest de l'Allemagne, en Rhénanie-Palatinat. C'est la commune de Paderborn qui a été principalement touchée, et les dégâts sont considérables.Réfugiés sous un abribus, ils sont pris par surprise par la tornade. Des vents à plus de 200 km/h balaient le centre-ville de Paderborn, vendredi 20 mai, dans l'ouest de l'Allemagne. Les habitants se barricadent en urgence car, dehors, les débris pleuvent, et les arbres s'écroulent. Un phénomène rarissime dans le pays, survenu après plusieurs jours de forte chaleur. Dans la ville de Lippstadt, une autre tornade se forme. La personne qui filme les images s'éloigne juste à temps. Quelques minutes plus tard, le magasin est entièrement dévasté. Des dégâts matériels de plusieurs milliers d'euros Scène de chaos après le passage de la tornade, les riverains sont abasourdis. "C'est un scénario de film de guerre", dit

In [124]:
scores_df = pd.DataFrame(scores)

In [125]:
scores_df

,title,description,climat,climat_score
0,Allemagne : une tornade spectaculaire a balayé l'ouest du pays,"Une tornade spectaculaire a balayé, vendredi 20 mai, l'ouest de l'Allemagne, en Rhénanie-Palatinat. C'est la commune de Paderborn qui a été principalement touchée, et les dégâts sont considérables.Réfugiés sous un abribus, ils sont pris par surprise par la tornade. Des vents à plus de 200 km/h balaient le centre-ville de Paderborn, vendredi 20 mai, dans l'ouest de l'Allemagne. Les habitants se barricadent en urgence car, dehors, les débris pleuvent, et les arbres s'écroulent. Un phénomène rarissime dans le pays, survenu après plusieurs jours de forte chaleur. Dans la ville de Lippstadt, une autre tornade se forme. La personne qui filme les images s'éloigne juste à temps. Quelques minutes plus tard, le magasin est entièrement dévasté. Des dégâts matériels de plusieurs milliers d'euros Scène de chaos après le passage de la tornade, les riverains sont abasourdis. ""C'est un scénario de film de guerre"", dit une habitante. Des arbres ont même été déracinés par la tempête. En tout, 43 personnes ont été blessées, et les dégâts matériels sont évalués à plusieurs millions d'euros. Ces intempéries record ont touché tout l'ouest du pays. Un homme de 38 ans est décédé.",1,C
1,Tornades : plus fréquentes et plus violentes avec le réchauffement climatique ?,"La tornade, provoquée par la rencontre de l'air chaud et de l'air froid, est encore un phénomène rare en Europe. Mais ce phénomène extrême se multipliera-t-il à l’avenir avec le réchauffement climatique ?C’est un spectacle effrayant que l’on observe aujourd’hui plus de six mois par an dans tout le centre des États-Unis. Des tornades d’une puissance parfois exceptionnelle comme l’une qui a lieu samedi 21 mai dans le Michigan et qui ne laisse derrière elle que des toitures arrachées et des voitures retournées. Ces scènes habituelles en Amérique du Nord pourraient-elles aussi le devenir en Europe, dès le printemps ? C’était impensable auparavant mais les tornades en République Tchèque l’année dernière et en Allemagne cette semaine ont changé la donne. Les climatologues divisés Pour que des nuages d’orage engendrent des tornades, il faut la rencontre de masse d’air chaud au sol et d’air beaucoup plus froid en altitude, poussé par des vents tournoyants. Ces phénomènes deviendront-ils plus nombreux et plus violents avec le dérèglement climatique ? La question fait encore débat chez les climatologues. ""En théorie, on devrait avoir des oranges plus violentes, des conditions convectives plus violentes et donc des situations plus favorables aux tornades. Cela, c’est la théorie pas encore observée"", explique Robert Vautard, climatologue et membre du Giec.",1,A
2,"Sécheresse : en Ardèche, départs d'incendies précoces pour la saison","En France, on totalise en ce printemps presque 40 jours consécutifs au-dessus des normales de saison, avec pour conséquence notamment une sécheresse précoce, en région Rhône-Alpes et en Ardèche particulièrement. Les premiers départs de feu ont été enregistrés.Une reprise de feu et un combat acharné contre les flammes. Depuis plus de 24 heures, les pompiers d'Ardèche sont sur le front. Deux incendies se sont déclarés dans le département. Des feux de forêt inhabituels pour un mois de mai. Les pompiers redoutent un été à haut risque. Les départs d'incendie sont très précoces pour la saison. Pour les habitants, le phénomène interroge. Crainte d'incendies à répétition Depuis le mois de janvier, les épisodes pluvieux sont rares, ils ont baissé de 40 % par rapport à l'an dernier. ""C'est inquiétant pour la flore, pour la faune également qui va souffrir, et puis pour les activités touristiques ça va être compliqué"", s'inquiète Didier Nury, maire (SE) de Maurac-en-Vivarais. Des rivières asséchées et la crainte d'incendies à répétition. La préfecture a imposé des restrictions d'eau à toute une partie du département de l'Ardèche.",1,B
3,Aude : deux morts par noyade à Narbon

In [65]:
len(scores_df)

8

In [66]:
scores_df.climat.mean()

0.625

In [68]:
scores_df.climat_score.value_counts()

C    3
D    2
Name: climat_score, dtype: int64

In [132]:
# meta TF1

In [133]:
meta = {"@context":"https://schema.org","@type":"WebSite","name":"TF1 Info","url":"https://www.tf1info.fr","publisher":{"@type":"NewsMediaOrganization","name":"TF1 Info","alternateName":"Télévision Française 1 Info","parentOrganization":{"@type":"Organization","name":"Groupe TF1","foundingDate":"1975-01-06","url":"https://groupe-tf1.fr/"},"url":"https://www.tf1info.fr","foundingDate":"2022-01-24","logo":{"@type":"ImageObject","url":"https://www.tf1info.fr/images/tf1info-logo-line.png","width":"155","height":"60"},"address":{"@type":"PostalAddress","streetAddress":"1 Quai du Point du Jour","addressLocality":"Boulogne-Billancourt","postalCode":"92100","addressCountry":"FR"},"sameAs":["https://www.facebook.com/TF1Info/","https://twitter.com/TFIINFO","https://news.google.com/publications/CAAqBwgKMNDhsQEw-fUJ","https://www.wikidata.org/wiki/Q111191650","https://www.youtube.com/channel/UCsrPUA0ZSDCNZC6wyRlR7ZA","https://www.tiktok.com/@tf1info","https://www.instagram.com/tf1info/"],"actionableFeedbackPolicy":"https://www.tf1info.fr/la-mediatrice-vous-repond/","correctionsPolicy":"https://www.tf1info.fr/la-mediatrice-vous-repond/","publishingPrinciples":"https://www.tf1info.fr/tf1-info/mentions-legales/","verificationFactCheckingPolicy":"https://www.tf1info.fr/actualite/les-verificateurs-12325/"}}

In [134]:
meta.keys()

dict_keys(['@context', '@type', 'name', 'url', 'publisher'])

In [135]:
meta["@context"]

'https://schema.org'